In [1]:
print("all ok")

all ok


In [2]:
from typing import List, Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [3]:
model_client=OpenAIChatCompletionClient(model='gpt-4o')

In [4]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."





In [4]:
def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [5]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from autogen_ext.tools.http import HttpTool

os.environ["SERPER_API_KEY"]=os.getenv("SERPER_API_KEY")
search_tool=GoogleSerperAPIWrapper(type='search')
def search_web(query : str)-> str:
    """Search the web with the given query and provide the results."""
    try:
        result=search_tool.run(query)
        return result
    except Exception as e:
        print(f"Error occured while searching the web: {e}")
        return "No result found"

In [6]:


planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself. Do not calculate anything delegate to DaDataAnalystAgent for calculation.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    #tools=[search_web_tool],
    tools=[search_web],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_web - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)


In [16]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=20)
termination = text_mention_termination | max_messages_termination


In [8]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""


In [9]:
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)


In [10]:
task = """Who was the Miami Heat player with the highest points in the 2006-2007 season, 
 what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"""


In [11]:
# Use asyncio.run(...) if you are running this in a script.
await team.reset()
await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, 
 what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address your inquiry, let's break it down into two main tasks:

1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. Calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 NBA seasons.

Here are the steps to obtain the necessary information:

1. WebSearchAgent: Research and identify the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Find total rebounds for this player during the 2007-2008 NBA season.
3. WebSearchAgent: Find total rebounds for this player during the 2008-2009 NBA season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds for this player between the 2007-2008 and 2008-2009 s

TaskResult(messages=[TextMessage(id='bd53b096-3221-4ceb-9b62-6a874f3f83e3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 24, 18, 51, 13, 746060, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, \n what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='5f0664a2-da9e-4aab-80f5-ee78046c197e', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=175, completion_tokens=191), metadata={}, created_at=datetime.datetime(2025, 7, 24, 18, 51, 23, 203913, tzinfo=datetime.timezone.utc), content="To address your inquiry, let's break it down into two main tasks:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. Calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 NBA seasons.\n\nHere are the steps to obtain the necessary 

### Custom Selector Function

In [12]:
task="""Which party won the 2021 election in west bengal.
        what was the improvement percentage based on 2016 election in west bengal"""

In [13]:
def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Which party won the 2021 election in west bengal.
        what was the improvement percentage based on 2016 election in west bengal


---------- TextMessage (PlanningAgent) ----------
To answer your questions, we'll need to find out the election results for both 2016 and 2021 in West Bengal and then calculate the improvement percentage. Here is the breakdown of tasks:

1. WebSearchAgent: Find out which party won the 2021 election in West Bengal.
2. WebSearchAgent: Find out what was the performance of the winning party in the 2016 West Bengal election.
3. DataAnalystAgent: Calculate the improvement percentage of the winning party from 2016 to 2021 based on the number of seats or vote share improvements.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_vmNs6MhKjtZSFHyfxZkRJjOX', arguments='{"query": "West Bengal 2021 election results winning party"}', name='search_web'), FunctionCall(id='call_uxd4cQ9lJPAbDFS2pnoXsvNW', arguments='{"query": "West Bengal 2016 election results Trinamool Congress performance"}', name='search_web')]
---------- ToolCallExecutionEvent (WebSearchAgent) -------

TaskResult(messages=[TextMessage(id='28b0af59-eaa6-47dd-a1d3-b3a58d16b41a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 24, 18, 52, 22, 471152, tzinfo=datetime.timezone.utc), content='Which party won the 2021 election in west bengal.\n        what was the improvement percentage based on 2016 election in west bengal', type='TextMessage'), TextMessage(id='b0e2a709-2bf2-4a7d-bb29-06f143d614db', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=157, completion_tokens=118), metadata={}, created_at=datetime.datetime(2025, 7, 24, 18, 52, 28, 188329, tzinfo=datetime.timezone.utc), content="To answer your questions, we'll need to find out the election results for both 2016 and 2021 in West Bengal and then calculate the improvement percentage. Here is the breakdown of tasks:\n\n1. WebSearchAgent: Find out which party won the 2021 election in West Bengal.\n2. WebSearchAgent: Find out what was the performance of the winning party in the 2016 W

### Custom Candidate Function

In [14]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    candidate_func=candidate_func,
)

await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Which party won the 2021 election in west bengal.
        what was the improvement percentage based on 2016 election in west bengal
---------- TextMessage (PlanningAgent) ----------
To determine which party won the 2021 West Bengal election and calculate the improvement percentage based on the 2016 election, we need to follow these steps:

1. WebSearchAgent: Search for the political party that won the 2021 West Bengal election.
2. WebSearchAgent: Search for the number of seats won by that party in the 2016 West Bengal election.
3. WebSearchAgent: Search for the number of seats won by that party in the 2021 West Bengal election.
4. DataAnalystAgent: Calculate the improvement percentage in seats won by the party from the 2016 election to the 2021 election.

Let's begin with these tasks: 

1. WebSearchAgent: Search for the political party that won the 2021 West Bengal election.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCa

TaskResult(messages=[TextMessage(id='37cb0795-f2d2-4cb3-9290-f874ff00d032', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 24, 19, 1, 2, 790085, tzinfo=datetime.timezone.utc), content='Which party won the 2021 election in west bengal.\n        what was the improvement percentage based on 2016 election in west bengal', type='TextMessage'), TextMessage(id='9d9c0a15-f3aa-4bd0-b6ab-09d27848614e', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=157, completion_tokens=161), metadata={}, created_at=datetime.datetime(2025, 7, 24, 19, 1, 15, 367218, tzinfo=datetime.timezone.utc), content="To determine which party won the 2021 West Bengal election and calculate the improvement percentage based on the 2016 election, we need to follow these steps:\n\n1. WebSearchAgent: Search for the political party that won the 2021 West Bengal election.\n2. WebSearchAgent: Search for the number of seats won by that party in the 2016 West Bengal election.\n3. 

### User Proxy Agent

In [17]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", 
                                  description="A proxy for the user to approve or disapprove tasks.",
                                  input_func=input)


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Which party won the 2021 election in west bengal.
        what was the improvement percentage based on 2016 election in west bengal


---------- TextMessage (PlanningAgent) ----------
To answer the query, we need to perform the following tasks:

1. Find out which party won the 2021 election in West Bengal.
2. Find the election results of the 2016 election in West Bengal to calculate the percentage improvement.
3. Calculate the percentage improvement from 2016 to 2021 for the winning party.

Here are the tasks that need to be executed:

1. WebSearchAgent: Search for the party that won the 2021 West Bengal election.
2. WebSearchAgent: Search for the election results of West Bengal in 2016, focusing on the winning party's results.
3. DataAnalystAgent: Calculate the improvement percentage of the winning party based on results from 2016 and 2021 in West Bengal.

Let's proceed with these tasks in this order.
---------- TextMessage (UserProxyAgent) ----------
APPROVE
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_n3kWAZMfYajgSCzRoUdbqBDi', arguments='{"query": "party that won 2021 electio

TaskResult(messages=[TextMessage(id='8b6b459f-d5fc-4379-9059-46436bc3cc24', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 24, 19, 14, 47, 896578, tzinfo=datetime.timezone.utc), content='Which party won the 2021 election in west bengal.\n        what was the improvement percentage based on 2016 election in west bengal', type='TextMessage'), TextMessage(id='1ac017c2-23d8-4d05-a4a3-00124473436f', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=157, completion_tokens=165), metadata={}, created_at=datetime.datetime(2025, 7, 24, 19, 14, 51, 662356, tzinfo=datetime.timezone.utc), content="To answer the query, we need to perform the following tasks:\n\n1. Find out which party won the 2021 election in West Bengal.\n2. Find the election results of the 2016 election in West Bengal to calculate the percentage improvement.\n3. Calculate the percentage improvement from 2016 to 2021 for the winning party.\n\nHere are the tasks that need to be exe

#### Using Reasoning Models

In [18]:
model_client = OpenAIChatCompletionClient(model="o3-mini")

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web],
    model_client=model_client,
    system_message="""Use web search tool to find information.""",
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""Use tool to perform calculation. If you have not seen the data, ask for it.""",
)

user_proxy_agent = UserProxyAgent(
    "UserProxyAgent",
    description="A user to approve or disapprove tasks.",
)

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
When the task is complete, let the user approve or disapprove the task.
"""

team = SelectorGroupChat(
    [web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,  # Use the same termination condition as before.
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
)


In [ ]:
await team.reset()
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Which party won the 2021 election in west bengal.
        what was the improvement percentage based on 2016 election in west bengal
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_C4Q6BLU9b2YVplz6viuEoiPH', arguments='{"query": "2021 West Bengal election winner and improvement percentage compared to 2016 West Bengal election result"}', name='search_web')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content="The 2021 West Bengal Legislative Assembly election was the 17th quinquennial legislative election held in West Bengal, to elect all 294 members of West ... TMC upped its vote share substantially in phases 7 and 8 that were held in late April. Polls during these phases were held in the Greater ... India News: West Bengal chief minister Mamata Banerjee-led Trinamool Congress (TMC) on Sunday beat the BJP in the assembly elections with a ... In the 2021 West Bengal elec